# Notebook 2: Relationale Algebra

## Aufgabe 1: Relationale Algebra in Python (6 Punkte)

In dieser Aufgabe sollt ihr selbst zwei abgeleitete Operatoren und den Grouping-Operator der relationalen Algebra implementieren.

In unserer Implementierung erlauben wir im Gegensatz zur allgemeinen Definition aus der Vorlesung keine gleichnamigen Attribute. Dadurch müssen wir uns nicht um mehrdeutige Attributnamen beim kartesischen Produkt (oder abgeleiteten Operatoren) kümmern und keine Punkt-Notation zum Referenzieren von Attributen verwenden.

Als Basis für unsere Implementierung dient die Klasse `Relation` aus der `relation.py`. Sie beschreibt eine Relation mit Name, Schema (bestehend aus Attributnamen und Domains) und eine Menge von Tupeln. Für euch sind insbesondere die folgenden Methoden der Klasse wichtig:
* `__init__`: Der Konstruktor bildet aus einem Namen `name` und ein Schema `schema` eine neue **leere** Relation.
* `add_tuple`: Fügt der Relation ein Tupel `tup` hinzu falls das Schema übereinstimmt
* `set_name`: Ändert den Namen eines Relationsobjekts in `name` um. Im Unterschied zu `renaming_relation` wird kein neues Relationsobjekt erzeugt. Die Umbenennung findet also *in-place* statt. Diese Methode ist wichtig um bei der Implementierung abgeleiteter Operatoren, den richtigen Relationsnamen zu setzen.
* `print_table`: Gibt die Relation in tabellarischer Form aus.
Außerdem hat Relation die folgenden Hilfsmethoden, die beim Grouping-Operator benötigt werden:
* `has_attribute`: Gibt für ein Attributnamen `attribute` den Wert `True` zurück, wenn die Relation ein Attribut mit diesem Namen besitzt.
* `get_attribute_domain`: Gibt für ein Attributnamen `attribute` des Typen zurück, wenn die Relation ein Attribut mit diesem Namen besitzt.
* `get_attribute_index`: Gibt für ein Attributnamen `attribute` die Position zurück, an der dieses Attribut im Schema der Relation steht, wenn die Relation ein Attribut mit diesem Namen besitzt.

Alle anderen Methoden, die die Klasse bereitstellt, sollten bei der Bearbeitung der Aufgabe keine Rolle spielen, sind aber auch dokumentiert.

Zunächst importieren wir die Klasse `Relation` und alle Basisoperatoren aus der Vorlesung in dieses Notebook, indem wir mithilfe von Cell Magic das Skript `relation.py`, in dem diese definiert sind, ausführen.

In [ ]:
%run relation  # runs the relation.py and thereby imports the Relation class and predefined base operators

Anschließend lesen wir alle Relationen aus dem PhotoDB Schema über die zur Verfügung gestellten .csv-Dateien ein. Hierzu wird die Funktion `load_csv` verwendet, die jeweils ein fertiges Relationsobjekt für jede .csv-Datei erstellt.

In [ ]:
# load .csv files into relation objects
erfahrungsstufen = load_csv('photodb/erfahrungsstufen.csv', 'erfahrungsstufen')
fotoabzuege = load_csv('photodb/fotoabzuege.csv', 'fotoabzuege')
fotographen = load_csv('photodb/fotographen.csv', 'fotographen')
fotos = load_csv('photodb/fotos.csv', 'fotos')
kameras = load_csv('photodb/kameras.csv', 'kameras')
kunden = load_csv('photodb/kunden.csv', 'kunden')
managed = load_csv('photodb/managed.csv', 'managed')
mitarbeiter = load_csv('photodb/mitarbeiter.csv', 'mitarbeiter')
personen = load_csv('photodb/personen.csv', 'personen')
seniors = load_csv('photodb/seniors.csv', 'seniors')
verkaufen = load_csv('photodb/verkaufen.csv', 'verkaufen')
verkaeufer = load_csv('photodb/verkaeufer.csv', 'verkaeufer')

Um den Einstieg etwas zu erleichtern, schauen wir uns zunächst die Implementierung des abgeleiteten Schnitt-Operators (intersection) an. Unsere Implementierung sieht wie folgt aus:

```python
def intersection(relation1, relation2):
    """
    Computes the intersection of two relations

    :param relation1: the first relation
    :param relation2:  the second relation
    :return: a new relation representing the intersection of the given relations
    """
    # integrity checks
    assert relation1.attributes == relation2.attributes  # the schema of both relations has to be identical
    assert relation1.domains == relation2.domains  # # the schema of both relations has to be identical
    # create empty new relation
    new_name = '(' + relation1.name + ') ∩ (' + relation2.name + ')'
    new_schema = build_schema(relation1.attributes, relation1.domains)
    new_relation = Relation(new_name, new_schema)
    # add all tuples of the intersection to the new relation
    for tup in relation1.tuples & relation2.tuples:
        new_relation.add_tuple(tup)
    return new_relation
```

Die Funktion ist folgendermaßen aufgebaut:
* Als Argumente bekommt sie zwei Relationen `relation1` und `relation2`, deren Schnitt berechnet werden soll.
* Da der Schnitt voraussetzt, dass die Schemata der beiden Relationen gleich sein müssen, testen wir dies zunächst mit zwei Assertions. Dabei testet der erste Ausdruck die Gleichheit der Attributnamen beider Relationen und der zweite die Gleichheit der Typen (oder Domains). Intern verwaltet ein Relationsobjekt sein Schema als ein Tupel von Attributnamen (`attributes`) und ein Tupel von Typen zu den entsprechenden Attributnamen (`domains`).
* Anschließend wird eine neue, leere Relation erstellt. Zum Erstellen einer Relation benötigen wir einen Namen (`new_name`) und das Schema (`new_schema`) der Ergebnisrelation. Als Namen wählen wir die Operation (Schnitt) in Verbindung mit den Namen der beiden Eingaberelationen. Da die Schemata beider Eingaberelationen gleich sind, können wir einfach eines der Schemata übernehmen. Hierzu verwenden wir die Hilfsfunktion `build_schema`, die aus einem Tupel aus Attributnamen und einem Tupel aus Typen ein Schema, wie es vom Konstruktor erwartet wird, erstellt. Die leere Relation wird dann durch Aufrufen des Konstruktors `Relation` mit den beiden Parametern `new_name` und `new_schema` erstellt.
* Zum Schluss müssen noch die Tupel in die Relation eingefügt werden, die Teil der Ergebnisrelation sind. Intern verwaltet die Klasse `Relation` Tupel in einem `set` mit dem Namen `tuples`, das automatisch Doppelvorkommen eliminiert. Da die grundlegenden Mengenoperationen auf `sets` in Python vordefiniert sind, reicht es den Schnitt der beiden Tupelsets mit dem `&` Operator zu bilden und alle resultierenden Tupel in einer `for`-Schleife mit `add_tuple` in die Ergebnisrelation `new_relation` einzufügen und anschließend zurückzugeben.

Im Folgenden soll der Theta-Join, der Left-Semi-Join und Grouping mit Aggregation implementiert werden. Beim Setzen des Namens der Ergebnisrelation könnt ihr das Symbol für die Operation einfach aus dem Erwarteten Output der Tests kopieren. Bei den Tests kann es aufgrund der Verwaltung von Tupeln als `sets` sein, dass die Reihenfolge eures Ergebnisses eine andere ist.

### 1.1 Theta Join (1.5 Punkte)

Implementiere den Theta-Join-Operator. Dabei sind `relation1` und `relation2` Objekte der Klasse `Relation` und `theta` ein Prädikat in Form eines strings, wie es auch bei der Selektion und dem dazugehörigen Operator `selection` verwendet wird. Zwei Beispiele zur Verwendung des Operators mit erwartetem Ergebnis findet ihr in der Testzelle.

In [ ]:
# THETA JOIN
def theta_join(relation1, relation2, theta):
    """
    Performs a theta join between two relations by first performing the cartesian product and then selection

    :param relation1: the first relation
    :param relation2: the second relation
    :param theta: the theta defining the join predicate
    :return: a new relation computed by executing the theta join
    """
    # TODO: implement this function
    pass

In [ ]:
# 1.5 Punkte
# Grading Test:
# theta_join(fotos, renaming_attributes(kameras, ['kid<-id']), 'kamera==kid').print_table()  # Zu jedem Foto, die Informationen über die Kamera, mit der sie aufgenommen wurden
# expected output:
#----------------------------------------------
#(fotos) ⋈_{kamera==kid} (ρ_{kid<-id}(kameras)) 
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#id                             gpskoordinaten                 aufnahmezeitpunkt              fotograph                      kamera                         kid                            marke                          modell                          
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#3                              (615.65,691.95)                2013-05-14 15:42:33.031867+02  7                              2                              2                              Canon                          650d                           
#1                              (123,234)                      2013-05-14 15:40:47.161073+02  3                              1                              1                              Canon                          6d                             
#2                              (3123.98,67.987)               2013-05-14 15:40:47.170802+02  6                              3                              3                              Nikon                          D3200                          
# projection(theta_join(personen, renaming_attributes(fotos, ['fotoid<-id']), 'id==fotograph'), ['id', 'name', 'vorname','fotoid']).print_table()  # Zu jeder FotoID den Namen und Vornamen des Fotographen, der das Foto geschossen hat
# expected output:
#--------------------------------------------------------------------------------
#π_{id,name,vorname,fotoid}((personen) ⋈_{id==fotograph} (ρ_{fotoid<-id}(fotos))) 
#--------------------------------------------------------------------------------
#id       name     vorname  fotoid    
#--------------------------------------------------------------------------------
#3        Mueller  Peter    1        
#7        Miese    Peter    3        
#6        Wurst    Hans     2 

### 1.2 Left Semi Join (1.5 Punkte)

Implementiere den Left-Semi-Join-Operator. Dabei sind `relation1` und `relation2` Objekte der Klasse `Relation` und `theta` ein Prädikat in Form eines strings, wie es auch bei der Selektion und dem dazugehörigen Operator `selection` verwendet wird. Zwei Beispiele zur Verwendung des Operators mit erwartetem Ergebnis findet ihr in der Testzelle.

In [ ]:
# LEFT SEMI JOIN
def left_semi_join(relation1, relation2, theta):
    """
    Perfoms a left semi join between two relations

    :param relation1: the first relation
    :param relation2: the second relation
    :param theta: The theta defining the join predicate
    :return: a new relation computed by executing the left semi join
    """
    # TODO: implement this function
    pass

In [ ]:
# 1.5 Punkte
# Grading Test:
# left_semi_join(fotographen, managed, 'mitarbeiterid==fotographid').print_table()  # Fotographen, die von Seniors gemanagt werden
# expected output:
#------------------------------------------------------
#(fotographen) ⋉_{mitarbeiterid==fotographid} (managed) 
#------------------------------------------------------
#mitarbeiterid   
#------------------------------------------------------
#7              
#3              
#6
# left_semi_join(personen, fotographen, 'id==mitarbeiterid').print_table()  # Persönliche Daten aller Fotographen
# expected output:
#----------------------------------------------
#(personen) ⋉_{id==mitarbeiterid} (fotographen) 
#--------------------------------------------------------
#id            name          vorname       geburtsdatum   
#--------------------------------------------------------
#7             Miese         Peter         1983-05-06    
#3             Mueller       Peter         1963-10-09    
#6             Wurst         Hans          1974-02-01   

### 1.3 Grouping mit Aggregation (3 Punkte)

In dieser Aufgabe soll der Grouping-Operator mit Aggregation implementiert werden. Dabei ist `relation` ein Objekt der Klasse `Relation`, `group` eine liste von Attributnamen nach denen gruppiert werden soll und `aggregations` eine Liste von Aggregationen.

Um die Implementierung zu erleichtern verwenden wir für die Aggregationen ausschließlich die folgenden Built-in Funktionen von Python die jeweils für Listen definiert sind:
* `max`: berechnet das Maximum einer Liste von Zahlen,
* `min`: berechnet das Minimum einer Liste von Zahlen,
* `sum`: berechnet die Summe einer Liste von Zahlen.

Ein Beispielaufruf der `grouping`-Funktion könnte folgendermaßen aussehen:
```python
grouping(managed, ['seniorid'], [(max, 'fotographid')])
```
Dabei gruppieren wir die relation `managed` nach dem Attribut `'seniorid'` und berechnen für jede Gruppe das Maximum des Attributs `'fotographid'`. Eine Gruppierung auf mehren Attributen und weitere Aggregationen wären ebenfalls möglich.

Geht bei der Implementierung wie folgt vor:
* **Gruppierung**: Erstellt zunächst die Gruppen. Eine einfache Möglichkeit ein Mapping von Gruppe auf alle Tupel der Gruppe zu verwalten ist die Verwendung eines dictionaries (`dict`), das ein Tupel (oder eine Liste) auf eine List von Tupeln der Gruppe abbildet.
* **Aggregation**: Berechnet anschließend für jede Gruppe jede der Aggregationen. Da die Built-in Funktionen von Python bereits für Listen definiert sind, reicht es zur Berechnung alle Werte eines Attributs einer Gruppe (beispielsweise `'fotographid'`) in eine Liste zu packen und die Built-in Funktion auf dieser Liste aufzurufen. Beim extrahieren der entsprechenden Attributwerte aus den Tupeln, helfen euch die Hilfsfunktionen, die eingangs beschrieben wurden.
* **Projektion**: Das Ergebnis der Gruppierung sind Tupel bestehend aus dem Teil, der die Gruppe beschreibt (beispielsweise `['seniorid']`) und den Werten der Aggregationen. Baut die Tupel entsprechend dieses Schemas zusammen.
* **Abschluss**: Zum Schluss müsst ihr ein neues Relationsobjekt erstellen. Der Name der Relation soll wie zuvor die Operation beinhalten. Beim Schema soll der Attributname für die Aggregationen aus der Funktion gefolgt von einem Unterstrich und dem Attributnamen, auf dem die Aggregation berechnet wurde, bestehen (besipielsweise `max_photographid`). Baut das Schema mithilfe der Funktion `build_schema`. Nach dem Initialisieren des neuen Relationsobjekts müssen nun noch die Ergebnistupel mittels `add_tuple` der Relation hinzugefügt werden.

Zwei Beispiele zur Verwendung des Operators mit erwartetem Ergebnis findet ihr in der Testzelle.

In [ ]:
# GROUPING
def grouping(relation, group, aggregations):
    """
    Performs grouping and aggregation on a relation

    :param relation: the relation
    :param group: a list of attributes the relation should be grouped by
    :param aggregations: a list of aggregations of the form (builtin_func, attribute) e.g. [(sum, 'photographid'), (len, 'fotographid')]
    :return: a new relation computed by executing grouping and computing the aggregations on the groups
    """
    # TODO: implement this function
    pass

In [ ]:
# 3.0 Punkte
# Grading Test:
# grouping(kameras, ['marke'], []).print_table()  # Kameras gruppiert nach Marke (keine Aggregation)
# expected output:
#------------------
#γ_{marke}(kameras) 
#------------------
#marke   
#------------------
#Canon  
#Nikon 
# grouping(managed, ['seniorid'], [(sum, 'fotographid'),(max, 'fotographid')]).print_table()  # Für jeden Senior, die Summe der FotographIDs und die maximale FotographID, der Fotographen, die sie managen (mit Aggregation)
# expected output:
#-------------------------------------------------------
#γ_{seniorid,sum(fotographid),max(fotographid)}(managed) 
#-------------------------------------------------------
#seniorid         sum_fotographid  max_fotographid   
#-------------------------------------------------------
#1                9                6                
#2                13               7                        

## Aufgabe 2: Übersetzung (3 Punkte)

In dieser Aufgabe sollen Anfragen in natürlicher Sprache in Ausdrücke der relationalen Alegbra und umgekehrt übersetzt werden. Als Grundlage dienen dazu die Relationen der Foto-Datenbank, die bereits in der Vorlesung und der vorherigen Aufgabe verwendet wurde. Nachfolgend sind noch mal die Schemata aller Relationen aufgelistet.

Unterstrichene Attribute sind dabei Primärschlüssel. Tupel können über dieses Attribut eindeutig identifiziert werden, d.h. es gibt keine zwei Tupel mit dem gleichen Wert für dieses Attribut.

Die Pfeile zeigen Fremdschlüsselbeziehungen an. Fremdschlüssel entsprechen dem Primärschlüssel einer anderen Relation und sind daher wichtig bei Joins. In der Relation *Kunden* zeigt die PersonID beispielsweise auf die Relation Personen und verweist damit auf das Tupel dieser Relation, mit dem es inhaltlich in Verbindung steht.

 $ \small {[}\text{Personen}{]}: \{{[} \underline{ID: int}, Name: string, Vorname: string, Geburtsdatum: date {]}\} $

$ \small {[}\text{Mitarbeiter}{]}: \{{[} \underline{PersonID: int \rightarrow Personen}, Gehalt: int, Erfahrung: int \rightarrow Erfahrungsstufen {]}\} $

$ \small {[}\text{Erfahrungsstufen}{]}: \{{[} \underline{ID: int}, Bezeichnung: string {]}\} $

$ \small {[}\text{Kunden}{]}: \{{[} \underline{PersonID: int \rightarrow Personen}, Branche: string {]}\} $

$ \small {[}\text{Seniors}{]}: \{{[} \underline{MitarbeiterID: int \rightarrow Mitarbeiter}, AnzahlGraueHaare: int, Bonus: int {]}\} $

$ \small {[}\text{Verkaeufer}{]}: \{{[} \underline{MitarbeiterID: int \rightarrow Mitarbeiter}, Spezialgebiet: string {]}\} $

$ \small {[}\text{Fotographen}{]}: \{{[} \underline{MitarbeiterID: int \rightarrow Mitarbeiter}{]}\} $

$ \small {[}\text{Fotoabzuege}{]}: \{{[} \underline{Foto: int \rightarrow Fotos, Zeitstempel: string}, Bildgroesse: int{]}\} $

$ \small {[}\text{Fotos}{]}: \{{[} \underline{ID: int}, GPSKoordinaten: point, Aufnahmezeitpunkt: string, Fotograph: int \rightarrow Fotographen,  Kamera: int \rightarrow  Kameras{]}\} $

$ \small {[}\text{Kameras}{]}: \{{[} \underline{ID: int}, Marke: string, Modell: string{]}\} $

$ \small {[}\text{Managed}{]}: \{{[} \underline{SeniorID: int \rightarrow Seniors, FotographID: int \rightarrow Fotographen, Von: date}, Bis: date{]}\} $

$ \small {[}\text{Verkaufen}{]}: \{{[} \underline{Kunde: int \rightarrow Kunden, Verkaeufer: int  \rightarrow Verkaeufer,} \underline{Foto: int \rightarrow Fotos}, Preis: int{]}\} $

### 2.1 Natürliche Sprache nach Relationale Alegbra (1.5 Punkte)

Übersetze die folgenden umgasprachlichen Ausdrücke in Ausdrücke der relationalen Alegbra. Bei komplexen Ausdrücken empfiehlt es sich Variablen für Teilausdrücke einzuführen. So könnte man beispielsweise einen Teilausdruck Angestellte wiefolgt einführen:

$$ \text{Angestellte} := \text{Personen} \ltimes \text{Mitarbeiter} $$

Bei der Bennennung der Variablen solltet ihr darauf achten, dass es noch keine Relation mit dem gleichen Namen gibt.
Den LaTeX-Befehl für alle wichtigen Operatoren findet ihr [hier](http://dbai.tuwien.ac.at/education/dm/resources/symbols.html).

* Die IDs aller Fotographen, die schon mehr als 100 Bilder gemacht haben.

$$ # TODO $$

In [ ]:
# 0.5 Punkte

* Vor- und Nachname aller Seniors, die einen höheren Bonus erhalten als die Summe der Gehälter aller Fotographen, die von ihnen gemanagt werden.

Seniors und die Summe der gehälter der Fotographen, die sie managed:
$$ # TODO $$

In [ ]:
# 1.0 Punkt

### 2.2 Relationale Algebra nach natürliche Sprache (1.5 Punkte)

In diesem Aufgabeteil soll die Übersetzung in die andere Richtung durchgeführt werden. Übersetze die nachfolgenden Ausdrücke der relationalen Algebra in natürliche Sprache. Versuche dabei präzise Formulierungen zu wählen.

* $ \pi_{Vorname, Geburtsdatum}(\sigma_{Geburtsdatum<01.01.2000} Personen) $

### TODO

In [ ]:
# 0.5 Punkte

* $ \pi_{ID}\Big (\sigma_{Erfahrung==ID \land Bezeichnung=='Fortgeschrittener'}(\text{Mitarbeiter} \times \text{Erfahrungsstufen})\Big) $

### TODO

In [ ]:
# 1.0 Punkte

## Aufgabe 3: Ausgabegröße (1 Punkt)

Sowohl beim Anwenden der Selektion als auch der Projektion auf eine Relation kann es sein, dass die Anzahl der Tupel in der Ergebnisrelation geringer ist als in der ursprünglichen Relation. Erkläre für beide Operatoren kurz, wie es dazu kommen kann.

### TODO

In [ ]:
# 1.0 Punkte

## Abgabe

Das Jupyter Notebook ist über das CMS bis zum dort angegebenen Abgabezeitpunkt (Donnerstag Morgen 10 Uhr) hochzuladen. Setzt vor dem Hochladen alle Outputs über `Kernel -> Restart & Clear Outputs` zurück. Ladet eine einzelne `.ipynb` Datei hoch, insbesondere sollen Datensätze nicht mit abgegeben werden. Tragt nachfolgend alle Teammitglieder mit Matrikelnummer ein.

Team:
- **hier Team eintragen:** Vorname Nachname (Matrikelnummer)
- **hier Team eintragen:** Vorname Nachname (Matrikelnummer)
- **hier Team eintragen:** Vorname Nachname (Matrikelnummer)